In [ ]:
! pip install -U "transformers[torch]" pypdf python-docx langdetect sentencepiece sacremoses

In [6]:
import gradio as gr
from transformers import pipeline
from pypdf import PdfReader
import docx
import os
from langdetect import detect
from huggingface_hub import HfApi, notebook_login

In [ ]:
summarizer = pipeline(task="summarization", model="facebook/bart-large-cnn")
translator_to_french = pipeline(
    task="translation_en_to_fr", model="Helsinki-NLP/opus-mt-en-fr"
)
translator_to_english = pipeline(
    task="translation_fr_to_en", model="Helsinki-NLP/opus-mt-fr-en"
)

MAX_FILE_SIZE = 10000000

In [ ]:
class TextExtractor:
    def __init__(self, doc_location: str):
        if doc_location is None:
            raise Exception(f"Please select a PDF to summarize")
        self.doc_location = doc_location

    def extract_text_from_pdf(self):
        reader = PdfReader(self.doc_location)
        text = ""

        for page in reader.pages:
            text += page.extract_text()

        return text

    def extract_text_from_doc(self):
        doc = docx.Document(self.doc_location)
        text = ""

        for paragraph in doc.paragraphs:
            text += paragraph.text + "\n"
        return text

    def extract_text_from_txt(self):
        with open(self.doc_location, "r", encoding="utf-8") as file:
            text = file.read()
        return text

    def extract_text_from_txt(self):
        with open(self.doc_location, "r", encoding="utf-8") as file:
            text = file.read()
        return text
    
    def text_length(self):
        words = self.text.split()
        num_words = len(words)
        return num_words

    def get_text(self) -> str:
        file_extension = os.path.splitext(self.doc_location)[1]
        if file_extension == ".pdf":
            self.text = self.extract_text_from_pdf()
        elif file_extension == ".txt":
            self.text = self.extract_text_from_txt()
        elif file_extension == ".docx" or file_extension == ".doc":
            self.text = self.extract_text_from_doc()
        else:
            raise gr.Error(f"We only support .pdf, .txt, .doc and .docx files")

        if len(self.text) > MAX_FILE_SIZE:
            raise gr.Error(
                f"Document exceeds the maximum supported size of {MAX_FILE_SIZE} characters."
            )

        return self.text

In [ ]:
text_extractor = TextExtractor("data/doc-file-example.docx")
text = text_extractor.get_text()

text_length = text_extractor.text_length()
summary_length = int(text_length / 2)

summary = summarizer(text, max_length=summary_length, do_sample=False)[0]["summary_text"]

In [ ]:
summary

In [ ]:
detected_lang = detect(summary)
detected_lang

## Update space

In [7]:
notebook_login()

In [4]:
api = HfApi()
api.upload_file(
    path_or_fileobj="requirements.txt",
    path_in_repo="requirements.txt",
    repo_id="noahnsimbe/text-summarizer",
    repo_type="space",
)
api.upload_file(
    path_or_fileobj="app.py",
    path_in_repo="app.py",
    repo_id="noahnsimbe/text-summarizer",
    repo_type="space",
)

RepositoryNotFoundError: 401 Client Error. (Request ID: Root=1-661c1ecf-52c64762759a25cc211cb2bf;f22fe883-c6ad-4499-9324-1d9cb49ce6b7)

Repository Not Found for url: https://huggingface.co/api/spaces/noahnsimbe/text-summarizer/preupload/main.
Please make sure you specified the correct `repo_id` and `repo_type`.
If you are trying to access a private or gated repo, make sure you are authenticated.
Invalid username or password.
Note: Creating a commit assumes that the repo already exists on the Huggingface Hub. Please use `create_repo` if it's not the case.